In [4]:
import vk_api
import datetime
import pytz
import json

tz = pytz.timezone('Europe/Moscow')


# Чтение токена из файла конфигурации
with open('../../../access/config.json') as config_file:
    config = json.load(config_file)

token = config['vk_token_group']

# создаем сессию API ВК
vk_session = vk_api.VkApi(token=token)
vk = vk_session.get_api()

# список всех собеседников в сообществе
dialogs = vk.messages.getConversations()
messages_count = dialogs['count']
print(f'Всего сообщений: {messages_count}')

count = 200
offset = 0

# Зацикливаю поиск, сдвигая offset
while offset < messages_count:
    # собираю идентификаторы собеседников
    dialogs = vk.messages.getConversations(count=count, offset=offset)['items']
    all_interlocutors = [user['conversation']['peer']['id'] for user in dialogs]

    # перебираем всех собеседников и получаем дату первого сообщения
    for interlocutors in all_interlocutors:
        history = vk.messages.getHistory(user_id=interlocutors, rev=1)
        first_contact = history['items'][0]['date']
        # Собираю по выбранному часовому поясу
        date = datetime.datetime.fromtimestamp(first_contact, tz=pytz.utc).astimezone(tz)
        # Вывожу результат
        print(f"vk.com/id{interlocutors}, {date.strftime('%Y-%m-%d, %H:%M:%S, %Z%z')}")
    offset = offset + count


Всего сообщений: 4
